<h2>Import</h2>

In [1]:
import evaluate
import vad_utils
import numpy as np
import wave
import scipy.io.wavfile
from python_speech_features import mfcc
from python_speech_features import logfbank

<h2>Make Train Set of MFCC</h2>

In [3]:
vad_label = vad_utils.read_label_from_file("./data/train_label.txt") #读取字典
flag = True
train_all_label = []
for key in vad_label: #遍历
    fs, audio = scipy.io.wavfile.read("./wavs/train/{name}.wav".format(name = key)) #读取音频    
    train_single_mfcc = mfcc(audio, samplerate=fs, winlen=0.032, winstep=0.010) #提取mfcc
    train_single_logfbank = logfbank(audio, samplerate=fs, winlen=0.032, winstep=0.010)
    train_single_label = vad_label[key] #标签
    while len(vad_label[key]) < train_single_mfcc.shape[0]:
        vad_label[key].append(0) #补齐
    if flag == True:
        train_all_mfcc = train_single_mfcc
        train_all_logfbank = train_single_logfbank
        flag = False
    else:
        train_all_logfbank = np.vstack((train_all_logfbank,train_single_logfbank))
        train_all_mfcc = np.vstack((train_all_mfcc, train_single_mfcc)) #合成总mfcc特征集
    train_all_label.extend(vad_label[key]) #合成总特征集
    print('finish: {name}'.format(name = key))
print(train_all_logfbank.shape)
print(train_all_mfcc.shape)
print(len(train_all_label))

finish: 100-122655-0035
finish: 1001-134707-0047
finish: 1001-134708-0032
finish: 1006-135212-0043
finish: 1006-135212-0101
finish: 101-126859-0017
finish: 101-126860-0032
finish: 1012-133424-0068
finish: 1018-133447-0021
finish: 1018-135221-0009
finish: 102-129232-0014
finish: 102-129232-0086
finish: 1025-75365-0009
finish: 1025-92820-0009
finish: 1027-125147-0002
finish: 1028-133393-0010
finish: 1028-133395-0007
finish: 103-1240-0008
finish: 103-1241-0002
finish: 1034-121119-0019
finish: 104-123365-0010
finish: 1040-133433-0057
finish: 1046-133224-0037
finish: 1046-133225-0042
finish: 1049-133454-0042
finish: 1050-134119-0036
finish: 1050-134121-0009
finish: 1051-133883-0054
finish: 1052-139308-0000
finish: 1053-132821-0004
finish: 1053-132850-0009
finish: 1053-289242-0032
finish: 1054-143005-0051
finish: 1054-143005-0108
finish: 1058-131113-0079
finish: 1060-134451-0063
finish: 1061-142358-0026
finish: 1061-152224-0001
finish: 1065-134593-0020
finish: 1065-134611-0005
finish: 1066-1

<h2>Make Test Set on Dev</h2>

In [8]:
vad_label_dev = vad_utils.read_label_from_file("./data/dev_label.txt") #读取字典
flag = True
train_all_label_dev = []
for key in vad_label_dev: #遍历
    fs, audio = scipy.io.wavfile.read("./wavs/dev/{name}.wav".format(name = key)) #读取音频    
    train_single_mfcc = mfcc(audio, samplerate=fs, winlen=0.032, winstep=0.010) #提取mfcc
    train_single_label = vad_label_dev[key] #标签
    while len(vad_label_dev[key]) < train_single_mfcc.shape[0]:
        vad_label_dev[key].append(0) #补齐
    if flag == True:
        train_all_mfcc_dev = train_single_mfcc
        flag = False
    else:
        train_all_mfcc_dev = np.vstack((train_all_mfcc_dev, train_single_mfcc)) #合成总mfcc特征集
    train_all_label_dev.extend(vad_label_dev[key]) #合成总特征集
    print('finish: {name}'.format(name = key))
print(train_all_mfcc_dev.shape)
print(len(train_all_label_dev))

finish: 1031-133220-0062
finish: 1031-133220-0091
finish: 1051-133881-0021
finish: 107-22885-0023
finish: 1084-145077-0022
finish: 1085-169444-0008
finish: 1085-169444-0040
finish: 1092-134562-0029
finish: 1097-136955-0031
finish: 111-126277-0016
finish: 1112-128138-0027
finish: 1124-134775-0023
finish: 1124-134775-0053
finish: 1124-134775-0085
finish: 1152-126549-0021
finish: 1152-126549-0051
finish: 1154-129975-0015
finish: 1154-129976-0019
finish: 119-129515-0001
finish: 1212-14653-0012
finish: 123-26848-0010
finish: 1230-139216-0003
finish: 1230-139216-0048
finish: 126-122467-0039
finish: 1263-141777-0003
finish: 1263-141777-0031
finish: 1280-143268-0019
finish: 1298-1054-0012
finish: 1335-27593-0002
finish: 1335-27593-0034
finish: 1347-135885-0025
finish: 5671-50453-0004
finish: 5671-50453-0037
finish: 569-126489-0025
finish: 57-121742-0018
finish: 5712-48848-0042
finish: 5717-94876-0019
finish: 5720-48302-0007
finish: 5740-97610-0017
finish: 5765-231845-0012
finish: 581-126957-00

<h2>Train GMM</h2>

In [15]:
from sklearn.mixture import GaussianMixture
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
#划分数据集
#train_all = np.concatenate((train_all_mfcc,train_all_logfbank), axis=1) 没用logfbank
X_train, X_test, y_train, y_test = train_test_split(train_all_mfcc, train_all_label, test_size=0.2, random_state=0)

#训练GMM模型
gmm = GaussianMixture(n_components=2, covariance_type='full', max_iter=3000)
gmm.fit(train_all_mfcc)
y_pred = gmm.predict(train_all_mfcc_dev)
'''
y_train = np.array(y_train)
y_pred = np.array(y_pred)
'''
'''
#修正01分类类别
if accuracy_score(train_all_label, y_pred) < 0.50:
    for i in range(len(y_pred)):
        y_pred[i] = 1 - y_pred[i]
'''

print('Misclassified samples:%d' % (train_all_label_dev != y_pred).sum())
print('Accuracy: %.2f' % accuracy_score(train_all_label_dev, y_pred))
print(evaluate.get_metrics(y_pred, train_all_label_dev))

Misclassified samples:158177
Accuracy: 0.78
(0.8444906194098819, 0.26558769413221484)


In [6]:
#其他特征
from sklearn.mixture import GaussianMixture
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
#划分数据集
train_all = np.concatenate((train_all_mfcc,train_all_logfbank), axis=1) 
X_train, X_test, y_train, y_test = train_test_split(train_all_logfbank, train_all_label, test_size=0.2, random_state=0)

#训练GMM模型
gmm = GaussianMixture(n_components=2, covariance_type='full', max_iter=3000)
gmm.fit(train_all)
y_pred = gmm.predict(train_all)
'''
y_train = np.array(y_train)
y_pred = np.array(y_pred)
'''
'''
修正01分类类别
if accuracy_score(train_all_label, y_pred) < 0.50:
    for i in range(len(y_pred)):
        y_pred[i] = 1 - y_pred[i]
'''

print('Misclassified samples:%d' % (train_all_label != y_pred).sum())
print('Accuracy: %.2f' % accuracy_score(train_all_label, y_pred))
print(evaluate.get_metrics(y_pred, train_all_label))

Misclassified samples:1261162
Accuracy: 0.75
(0.8178631993945024, 0.2876999060917386)


<h2>DNN</h2>

In [4]:
#接近随机猜测,训练过慢
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset

#转换数据为PyTorch张量
X = torch.tensor(train_all_mfcc, dtype=torch.float32)
y = torch.tensor(train_all_label, dtype=torch.float32)
#划分数据集
X_train_dnn, X_test_dnn, y_train_dnn, y_test_dnn = train_test_split(X, y, test_size=0.2, random_state=42)
#创建数据加载器
train_dataset = TensorDataset(X_train_dnn, y_train_dnn)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# 定义模型
class DNN(nn.Module):
    def __init__(self):
        super(DNN, self).__init__()
        self.fc1 = nn.Linear(13, 128)  
        self.fc2 = nn.Linear(128, 64)  
        self.fc3 = nn.Linear(64, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.sigmoid(self.fc3(x))
        return x

#初始化模型、损失函数和优化器
model = DNN()
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.1)

#训练模型
num_epochs = 10
for epoch in range(num_epochs):
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels.unsqueeze(1))
        loss.backward()
        optimizer.step()
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

#在测试集上评估模型
with torch.no_grad():
    outputs = model(X_test_dnn)
    predicted = (outputs >= 0.5).float()
    accuracy = (predicted == y_test_dnn.unsqueeze(1)).float().mean()
    print(f'Accuracy on test set: {accuracy.item():.4f}')


Epoch [1/10], Loss: 7.6923
Epoch [2/10], Loss: 23.0769
Epoch [3/10], Loss: 23.0769
Epoch [4/10], Loss: 7.6923
Epoch [5/10], Loss: 15.3846
Epoch [6/10], Loss: 30.7692
Epoch [7/10], Loss: 46.1538
Epoch [8/10], Loss: 30.7692
Epoch [9/10], Loss: 23.0769
Epoch [10/10], Loss: 15.3846
Accuracy on test set: 0.8132


In [5]:
print(evaluate.get_metrics(predicted.tolist(), y_test_dnn.tolist())) #打印dnn结果

(0.5, 0.999997579688748)


<h2>On Test Set</h2>

In [8]:
#用DNN做
import os
f_res = open("./result2.txt","w") #写入结果的文件
f_res.truncate(0) #清空文件
paths = os.walk(r"./wavs/test") #得到路径
for path, dir_lst, file_lst in paths:
    for file_name in file_lst: #遍历测试集的音频
        file_name = file_name.split(".")[0] #分离出文件名(不带后缀)
        fs, audio = scipy.io.wavfile.read("./wavs/test/{name}.wav".format(name = file_name))
        test_mfcc = mfcc(audio, samplerate=fs, winlen=0.032, winstep=0.010) #特征提取
        X = torch.tensor(test_mfcc, dtype=torch.float32) #转化tensor
        outputs = model(X) #放入模型
        prediction = outputs.squeeze().tolist()
        res = vad_utils.prediction_to_vad_label(prediction=prediction, threshold=0.5)
        f_res.write("{name} {result}".format(name = file_name, result = res)+"\n") #将结果写入result.txt(音频按名称已排序)
f_res.close() #关闭文件

In [6]:
#用GMM做
import os
f_res = open("./result2.txt","w") #写入结果的文件
f_res.truncate(0) #清空文件
paths = os.walk(r"./wavs/test") #得到路径
for path, dir_lst, file_lst in paths:
    for file_name in file_lst: #遍历测试集的音频
        file_name = file_name.split(".")[0] #分离出文件名(不带后缀)
        fs, audio = scipy.io.wavfile.read("./wavs/test/{name}.wav".format(name = file_name))
        test_mfcc = mfcc(audio, samplerate=fs, winlen=0.032, winstep=0.010)
        prediction = gmm.predict(test_mfcc)
        res = vad_utils.prediction_to_vad_label(prediction=prediction, threshold=0.5)
        f_res.write("{name} {result}".format(name = file_name, result = res)+"\n") #将结果写入result.txt(音频按名称已排序)
f_res.close() #关闭文件